In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
news_df = pd.read_csv('/content/drive/MyDrive/Kaggle Datasets/WELFake_Dataset.csv',index_col=0)

In [ ]:
news_df.head()

In [ ]:
news_df['label'].value_counts()

In [ ]:
sns.heatmap(news_df.isnull())

In [ ]:
news_df.columns.values

In [ ]:
news_df = news_df.dropna()

In [ ]:
label=['label']

In [ ]:
final_df = pd.get_dummies(news_df,columns=label,drop_first=True)

In [ ]:
final_df.head()

In [ ]:
X=news_df.drop('label',axis=1)
y=news_df['label']

In [ ]:
X.shape,y.shape

In [ ]:
y.value_counts()

In [ ]:
from tensorflow.keras.layers import Embedding,LSTM,Dense,Dropout,Bidirectional
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot

In [ ]:
voc_size=5000

In [ ]:
massages=X.copy()

In [ ]:
massages = massages[:10000]

In [ ]:
massages['title'][9]

In [ ]:
massages = massages.reset_index()

In [ ]:
import nltk
import re
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')

In [ ]:
nltk.download('wordnet')

In [ ]:
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
ps = PorterStemmer()
lem = WordNetLemmatizer()
corpus = []

In [ ]:
for i in range(0,len(massages)):
  print(i)
  review = re.sub('[^a-zA-Z]',' ',massages['title'][i])
  review = review.lower()
  review = review.split()

  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)

In [ ]:
for i in range(0,len(massages)):
  print(i)
  review = re.sub('[^a-zA-Z]',' ',massages['title'][i])
  review = review.lower()
  review = review.split()

  review = [lem.lemmatize(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)

In [ ]:
corpus[0]

In [ ]:
onehot_repr = [one_hot(words,voc_size) for words in corpus]
onehot_repr

In [ ]:
sent_len=20
embedded_docs = pad_sequences(onehot_repr,padding='pre',maxlen=sent_len)
embedded_docs

In [ ]:
vec_feature=50
model = Sequential()
model.add(Embedding(voc_size,vec_feature,input_length=sent_len))
model.add(LSTM(100))

model.add(Dense(1,kernel_initializer='glorot_uniform',activation='sigmoid'))

model.compile('adam','binary_crossentropy',['accuracy'])

In [ ]:
vec_feature=50
model1 = Sequential()
model1.add(Embedding(voc_size,vec_feature,input_length=sent_len))
model1.add(Bidirectional(LSTM(50)))

model1.add(Dense(1,kernel_initializer='glorot_uniform',activation='sigmoid'))

model1.compile('adam','binary_crossentropy',['accuracy'])

In [ ]:
model1.summary()

In [ ]:
y = y[:10000]

In [ ]:
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [ ]:
X_final.shape, y_final.shape

In [ ]:
X_final

In [ ]:
y_final

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [ ]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),
          epochs=10,batch_size=64)

In [ ]:
model1.fit(X_train,y_train,validation_data=(X_test,y_test),
          epochs=5,batch_size=128)

In [ ]:
losses = pd.DataFrame(model.history.history)
losses.plot()

In [ ]:
losses = pd.DataFrame(model1.history.history)
losses.plot()

In [ ]:
predictions = model.predict(X_test)
predictions = (predictions>0.5)

In [ ]:
predictions1 = model1.predict(X_test)
predictions1 = (predictions1>0.5)

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score

In [ ]:
print('Normal LSTM')
print(confusion_matrix(y_test,predictions))
print('----------------------------------')
print('Bidirectional LSTM')
print(confusion_matrix(y_test,predictions1))

In [ ]:
print('Normal LSTM')
print(classification_report(y_test,predictions))
print('----------------------------------------')
print('Bidirectional LSTM')
print(classification_report(y_test,predictions1))

In [ ]:
print('Normal LSTM')
print(accuracy_score(y_test,predictions))
print('-------------------------')
print('Bidirectional LSTM')
print(accuracy_score(y_test,predictions1))